# Create Imagery Layers

# Table of Contents

* [Introduction](#Introduction)
* [Prerequisites](#Prerequisites)
* [Image Mosaic](#Image-mosaic)
* [One image](#One-image)
* [Image collection](#Image-collection)
* [Conclusion](#Conclusion)
* [Supported raster types](#Supported-raster-types)

## Introduction

Hosted imagery layers can be used to manage, share, and analyze raster and imagery data in your organization. You can use a hosted imagery layer as an input to analysis tools, create custom workflows with raster functions, manage large collections of imagery, and include imagery layers in maps.<br>
In this guide, you will learn how to create imagery layers for your data quickly and easily using the ArcGIS API for Python.

## Prerequisites

If you are creating imagery layers, you must have your GIS (either ArcGIS Online or ArcGIS Enterprise >= 10.9) properly configured for raster analysis, and you must have privileges to create content and publish hosted imagery layers.<br>
For privilege information for ArcGIS Online, refer [here](https://doc.arcgis.com/en/arcgis-online/reference/roles.htm).<br>
For privilege information for ArcGIS Enterprise, refer [here](https://enterprise.arcgis.com/en/portal/latest/administer/windows/roles.htm).

For this functionality to work in ArcGIS Online, Azure library packages for Python (Azure SDK for Python - azure-storage-blob: 12.1<= version <=12.8) needs to be pre-installed. Refer here for more information https://docs.microsoft.com/en-us/azure/developer/python/azure-sdk-install

In [1]:
import arcgis
from arcgis.gis import GIS
from arcgis.raster.analytics import copy_raster, create_image_collection, list_datastore_content

# Log into ArcGIS Online or ArcGIS Enterprise, ensuring that you have the required privileges as mentioned above.
gis = GIS("home")

By using the copy_raster function you can create one imagery layer by mosaicking multiple images.

## Image mosaic

By using the [copy_raster] (https://developers.arcgis.com/python/api-reference/arcgis.raster.analytics.html#copy-raster) function you can create one imagery layer by mosaicking multiple images. This option supports generating a single imagery layer that is a mosaic of multiple images over space, multidimensional raster data, and many raster types that allow you to control how your satellite or aerial products are processed. Use this option for imagery layers where advanced metadata is required, such as for multiband pre-processed satellite scenes.

The copy_raster function can be used to create an image mosaic from local raster datasets. The following parameters can be configured to create an image mosaic:
* The path to the local raster data can be specified using the ```input_raster``` parameter.
* The name of the output imagery layer can be specified using the ```output_name``` parameter.
* Using the ```raster_type_name``` parameter, you can specify the raster type, which identifies metadata such as georeferencing, acquisition date, sensor type, and band wavelengths, along with a raster format. The supported raster types can be found in the [table](#Supported-raster-types)
  at the end of this guide.
* ```raster_type_params``` can be used to configure the options for your imagery layer. This parameter is accepted as a
  dictionary and can be utilized to configure the following properties:
    * The productType key can be used to specify the type of product included in your satellite imagery. Product types are
      typically identified by various processing levels associated with the specific sensor. The list of product
      types for each raster type can be found in the [table](#Supported-raster-types) at the end of this guide.
    * The processingTemplate key can be used to perform common imagery processing tasks, such as extracting specific bands,
      pansharpening, orthorectification, stretching, and other tasks.
      The processing templates that can be used depends on the raster type you selected. The list of processing templates for
      each raster type can be found in the [table](#Supported-raster-types) at the end of this guide.
    *  The pansharpenType key helps you specify the pansharpen type you prefer. Further, you can also specify the filter and
       band weights. This option is honored when the input raster type includes coincident panchromatic and multispectral
       imagery data.<br>Possible values include: "Mean", "IHS", "Brovey", "Esri", "Mean", "Gram-Schmidt".
    * The stretchType key can be used to stretch each raster item in the imagery layer. The value can be one of: 
      MinMax; <min\>; <max\>, PercentMinxMax; <MinPercent\>; <MaxPercent\>, StdDev; <NumberOfStandardDeviation\>.
    > Example: ```{"productType":"All","processingTemplate":"Pansharpen", "pansharpenType":"Gram-Schmidt",
      "filter":"SharpenMore", "pansharpenWeights":"0.85 0.7 0.35 1", "StretchType": "MinMax; 0; 100"}```
* You can also specify additional settings through the ```context``` parameter. The settings that can be configured are:
    * The output spatial reference to project the output raster.
    > Example: ```{"outSR": {"wkid":3857}}```
    * You can also specify whether to build footprints. You can choose to build footprints for your imagery layer such
      that only pixels within a given range are included. This will help eliminate null data value pixels resulting in seamless
      mosaics. If you choose to build footprints, you can also provide additional footprint settings.<br> 
      You can also choose to define a NoData value for pixels in the imagery layer. Pixels with the NoData value will display
      as transparent in the map.
     > Example: ```{"buildFootprints":True, "footprintsArguments":{"method":"RADIOMETRY","minValue":1,"maxValue":5,
       "shrinkDistance":50,"skipOverviews":True,"updateBoundary":True,
       "maintainEdge":False,"simplification":None,"numVertices":20,
       "minThinnessRatio":0.05,"maxSliverSize":20,"requestSize":2000,
       "minRegionSize":100},
       "defineNodata":True,
       "noDataArguments":{"noDataValues":[500],"numberOfBand":99,"compositeValue":True},
       "buildOverview":True}```
* The ```tiles_only``` parameter can be used to decide whether you would like the imagery layer to be a Tiled Imagery
  Layer or a Dynamic Imagery Layer. On ArcGIS Online, the default is Tiled Imagery Layer. This can be set to Dynamic Imagery
  Layer by setting the value of this parameter to False.<br>On ArcGIS Enterprise, this parameter is not honored and will always
  generate a Dynamic Imagery Layer.

In [2]:
input_raster_folder = r"C:\data\input_rasters"
mosaic_imagery_layer = copy_raster(input_raster=input_raster_folder,
                                   output_name="mosaic_imagery_layer",
                                   raster_type_name="Raster Dataset",
                                   gis=gis)

Below are some examples for satellite imagery for the Landsat and Pleiades satellites along with the appropriate  ```raster_type_name``` and ```raster_type_params```:

In [3]:
landsat_data = r"C:\data\Landsat15MSS"
landsat_imagery_layer = copy_raster(input_raster=landsat_data,
                                    output_name="landsat_imagery_layer",
                                    raster_type_name="Landsat 1-5 MSS",
                                    raster_type_params={"productType": "Level1", "processingTemplate": "Multispectral"},
                                    gis=gis)

In [4]:
pleiades_data = r"C:\data\Pleiades1"
pleiades_imagery_layer = copy_raster(input_raster=pleiades_data,
                                     output_name="pleiades_imagery_layer",
                                     raster_type_name="Pleiades-1",
                                     raster_type_params={"productType": "ORTHO", "processingTemplate": "Pansharpen"},
                                     tiles_only=False,
                                     gis=gis)

Further, on ArcGIS Enterprise you can also make use of the data in your datastore (fileShares, rasterStores, cloudStores) to create hosted imagery layers. More information on creating datastores can be found [here](https://enterprise.arcgis.com/en/server/latest/manage-data/windows/registering-your-data-with-arcgis-server-using-manager.htm).

You can view the content in your datastore using the list_datastore_content function.

In [5]:
datastore_path = "/rasterStores/RasterDataStore/sentinel_data_folder"
list_datastore_content(datastore_path, gis=gis)

{'/rasterStores/RasterDataStore/sentinel_data_folder': ['/rasterStores/RasterDataStore/sentinel_data_folder/L1C_T10SEJ_A006760_20180622T190432/',
  '/rasterStores/RasterDataStore/sentinel_data_folder/L1C_T10SEJ_A015697_20180624T190108/']}

In [6]:
sentinel_imagery_layer = copy_raster(input_raster=datastore_path,
                                     output_name="sentinel_imagery_layer",
                                     raster_type_name="Sentinel-2",
                                     raster_type_params={"productType": "All", "processingTemplate": "Multispectral"},
                                     context={"image_collection_properties": {"imageCollectionType": "Satellite"}, "byref":True},
                                     gis=gis)

## One image

The [copy_raster](https://developers.arcgis.com/python/api-reference/arcgis.raster.analytics.html#copy-raster) function also enables you to create an imagery layer from just one image. This option supports all supported raster and image formats, multidimensional raster data, and configuration with various source types and compression settings. Use this option for simple imagery layers where advanced metadata is not needed.

Here, the ```input_raster``` parameter can accept the path to the local raster dataset. Since the only raster type that is supported is the Raster Dataset type, we do not have to specify the ```raster_type_name``` parameter here.

In [7]:
input_raster_path = r"C:\data\input_raster.tif"
single_image_layer = copy_raster(input_raster=input_raster_path,
                                 output_name="output_imagery_layer",
                                 gis=gis)

On ArcGIS Enterprise, you can also use data from your datastore as described above in the Image mosaic section.

In [8]:
datastore_path = "/rasterStores/rasterstore"
list_datastore_content(datastore_path, gis=gis)

{'/rasterStores/rasterstore': ['/rasterStores/rasterstore/090161.tif',
  '/rasterStores/rasterstore/esa_ca.crf/',
  '/rasterStores/rasterstore/Hosted_amberg_test_crf_cr_021.crf/',
  '/rasterStores/rasterstore/Hosted_CopyRaster_WEO4G6.crf/',
  '/rasterStores/rasterstore/Hosted_crf_2_byref.crf/',
  '/rasterStores/rasterstore/Hosted_output_imagery_layer_1.crf/',
  '/rasterStores/rasterstore/Hosted_testPotentialSolarPowerPlant02.crf/']}

In [9]:
input_raster = "/rasterStores/rasterstore/090161.tif"
single_image_layer_1 = copy_raster(input_raster=input_raster,
                                   output_name="output_imagery_layer_1",
                                   context={"byref": True},
                                   gis=gis)

## Image collection

The [create_image_collection](https://developers.arcgis.com/python/api-reference/arcgis.raster.analytics.html#create-image-collection) function is another function that can be used to create hosted imagery layers on ArcGIS Online and ArcGIS Enterprise. This function will help you create an imagery layer that manages a collection of many images. It supports querying single images within the imagery layer that is made up of a collection of images. It further supports storing the imagery in the original source format.

The create_image_collection function can be used to create an imagery layer from multiple input rasters. The following parameters can be configured to create an image collection:

* The ```image_collection``` parameter helps you set the name of the image collection you would like to create.
* Using the ```input_rasters``` parameter, you can pass in the list of rasters to be used to create the image collection via
  the local raster data folder path.
* With the ```raster_type_name``` parameter you can specify the raster type which identifies and uses metadata such as georeferencing, acquisition date, sensor type, and band wavelengths. The supported raster types can be found in the [table](#Supported-raster-types) 
  at the end of this guide.
* The ```raster_type_params``` can be used to configure the options for your imagery layer. A detailed explanation of these is
  given under the Image mosaic section of this guide.
* The ```context``` parameter can be used to provide additional input parameters.
    * The image_collection_properties can be set through the context as:
    > ```{"image_collection_properties": {"imageCollectionType": "Satellite"}}```
    * Additionally, other settings such as buildFootprints and NoData values can be set here. More details on this are
      explained in the Image mosaic section of this guide.

In [10]:
landsat_data = r"C:\data\Landsat15MSS"
landsat_imagery_layer = create_image_collection(image_collection="landsat15_image_collection",
                                                input_rasters=landsat_data,
                                                raster_type_name="Landsat 1-5 MSS",
                                                raster_type_params={"productType": "Level1", "processingTemplate": "Multispectral"},
                                                gis=gis)

On ArcGIS Enterprise you can also make use of the data in your datastore (fileShares, rasterStores, cloudStores) to create hosted imagery layers. More information on creating datastores can be found [here](https://enterprise.arcgis.com/en/server/latest/manage-data/windows/registering-your-data-with-arcgis-server-using-manager.htm).


You can view the content in your datastore using the list_datastore_content function.

In [11]:
datastore_path = "/rasterStores/RasterDataStore/sentinel_data_folder"
list_datastore_content(datastore_path, gis=gis)

{'/rasterStores/RasterDataStore/sentinel_data_folder': ['/rasterStores/RasterDataStore/sentinel_data_folder/L1C_T10SEJ_A006760_20180622T190432/',
  '/rasterStores/RasterDataStore/sentinel_data_folder/L1C_T10SEJ_A015697_20180624T190108/']}

In [12]:
sentinel_imagery_layer = create_image_collection(image_collection="sentinel_image_collection",
                                                 input_rasters=datastore_path,
                                                 raster_type_name="Sentinel-2",
                                                 raster_type_params={"productType":"All","processingTemplate":"Multispectral"},
                                                 context={"image_collection_properties":{"imageCollectionType":"Satellite"},"byref":True},
                                                 gis=gis)

## Conclusion

In this guide, you've learned how to create imagery layers from your local raster data or data in your datastore using the copy_raster and create_image_collection functions. These functions provide a simple way of creating an imagery layer from a normal raster dataset, multidimensional datasets, and even satellite datasets. You have also seen how to create an imagery layer from either a single or multiple raster datasets.

Your imagery layers are now published and ready for further analysis. A comprehensive guide on using imagery layers can be found at https://developers.arcgis.com/python/guide/using-imagery-layers/.

<hr>

## Supported raster types

| Raster Type   | pansharpenWeights | productType | processingTemplate | 
| -------- | ----------- | ----------- | ----------- |
| "QuickBird" | "0.85, 0.7, 0.35, 1" | "All" (default), "Basic",<br>"Orthorectified",<br>"Standard Orthoready",<br>"Standard"<br><br><br><br><br><br>----------------------------------------<br>"Orthorectified AComp",<br>"Standard AComp",<br>"Standard OrthoreadyAComp" | "All Bands",<br>"Multispectral",<br>"Multispectral AComp",<br>"Panchromatic",<br>"Panchromatic AComp",<br>"Pansharpen",<br>"Pansharpen AComp",<br>"Pansharpen and Multispectral (default)"<br><br>-------------------------------------------<br>"Multispectral AComp (default)",<br>"Panchromatic AComp",<br>"Panshapen AComp"
| "DubaiSat-2" | "0.166,0.167,0.167,0.5" | "All" (default),<br> "Geo",<br> "Radio" | "All Bands",<br> "Multispectral",<br> "Panchromatic",<br> "Pansharpen",<br> "Pansharpen and Multispectral" (default) |
| "GF-1 PMS" | "0.2501,0.3646,0,0.3853" | "All" (default),<br> "LEVEL 1" | "All Bands",<br> "Multispectral",<br> "Panchromatic",<br> "Pansharpen",<br> "Pansharpen and Multispectral" (default) |
| "GF-1 WFV" | - | "All" (default),<br> "LEVEL 1" | "Multispectral" (default) |
| "GF-2 PMS" | "0.2501,0.3646,0,0.3853" | "All" (default),<br> "LEVEL 1" | "All Bands",<br> "Multispectral",<br> "Panchromatic",<br> "Pansharpen",<br> "Pansharpen and Multispectral" (default) |
| "KOMPSAT-2" | "0.166,0.167,0.167,0.5" | "All" (default),<br> "KOMPSAT-2 Scene 1A",<br> "KOMPSAT-2 Scene 2A" | "All Bands",<br> "Multispectral",<br> "Panchromatic",<br> "Pansharpen",<br> "Pansharpen and Multispectral" (default) |
| "KOMPSAT-3" | "0.166,0.167,0.167,0.5" | "All" (default),<br> "Level 1G",<br> "Level 1O" "Level 1R" | "All Bands",<br> "Multispectral",<br> "Panchromatic",<br> "Pansharpen",<br> "Pansharpen and Multispectral" <br>(default) |
| "Landsat 8" | "0.42, 0.51, 0.07, 0.0" | <br><br><br><br><br><br>"All" (default)<br><br><br><br><br><br><br>----------------------------------------<br><br><br><br><br>"Level1"<br><br><br><br><br>----------------------------------------<br>"Surface Reflectance<br><br>----------------------------------------<br>"Spectral Indices"<br><br>----------------------------------------<br>"Top of Atmosphere Reflectance"<br><br>----------------------------------------<br>"Brightness Temperature"<br><br>----------------------------------------<br>"Surface Temperature" | "All Bands",<br> "Brightness Temperature",<br> "Cirrus", "Cloud", "Landcover",<br> "Multispectral", "Panchromatic",<br> "Pansharpen",<br> "Pansharpen and Multispectral" <br>(default),<br>"QA", "SnowIce", "Spectral Indices",<br> "Surface Reflectance", "Thermal",<br> "Top of Atmosphere Reflectance",<br> "Vegetation", "Water",<br> "Surface Temperature"<br><br>----------------------------------------<br>"All Bands", "Cirrus",<br> "Cloud","Landcover",<br> "Multispectral", "Panchromatic",<br> "Pansharpen",<br> "Pansharpen and Multispectral" <br>(default),<br>"QA","SnowIce", "Thermal",<br> "Vegetation", "Water" <br><br>----------------------------------------<br> "Surface Reflectance"<br><br>----------------------------------------<br> "Spectral Indices"<br><br>----------------------------------------<br>"Top of Atmosphere Reflectance"<br><br>----------------------------------------<br>"Brightness Temperature"<br><br>----------------------------------------<br>"Surface Temperature"|
| "Landsat 7 ETM+" | "0.11, 0.14, 0.14, 0.61" | <br><br><br><br><br><br>"All" (default)<br><br><br><br><br><br><br>----------------------------------------<br><br><br><br><br>"Level1"<br><br><br><br><br>----------------------------------------<br>"Surface Reflectance<br><br>----------------------------------------<br>"Spectral Indices"<br><br>----------------------------------------<br>"Top of Atmosphere Reflectance"<br><br>----------------------------------------<br>"Brightness Temperature"<br><br>----------------------------------------<br>"Surface Temperature" | "All Bands",<br> "Brightness Temperature",<br> "Cirrus", "Cloud", "Landcover",<br> "Multispectral", "Panchromatic",<br> "Pansharpen",<br> "Pansharpen and Multispectral" <br>(default),<br>"QA", "SnowIce", "Spectral Indices",<br> "Surface Reflectance", "Thermal",<br> "Top of Atmosphere Reflectance",<br> "Vegetation", "Water",<br> "Surface Temperature"<br><br>----------------------------------------<br>"All Bands", "Cirrus",<br> "Cloud","Landcover",<br> "Multispectral", "Panchromatic",<br> "Pansharpen",<br> "Pansharpen and Multispectral" <br>(default),<br>"QA","SnowIce", "Thermal",<br> "Vegetation", "Water" <br><br>----------------------------------------<br> "Surface Reflectance"<br><br>----------------------------------------<br> "Spectral Indices"<br><br>----------------------------------------<br>"Top of Atmosphere Reflectance"<br><br>----------------------------------------<br>"Brightness Temperature"<br><br>----------------------------------------<br>"Surface Temperature"|
| "Landsat 4-5 TM" | - | <br><br><br><br><br><br>"All" (default)<br><br><br><br><br><br><br>----------------------------------------<br><br><br><br><br>"Level1"<br><br><br><br>----------------------------------------<br>"Surface Reflectance<br><br>----------------------------------------<br>"Spectral Indices"<br><br>----------------------------------------<br>"Top of Atmosphere Reflectance"<br><br>----------------------------------------<br>"Brightness Temperature"<br><br>----------------------------------------<br>"Surface Temperature" | "All Bands",<br> "Brightness Temperature",<br> "Cirrus", "Cloud", "Landcover",<br> "Multispectral", "Panchromatic",<br> "Pansharpen",<br> "Pansharpen and Multispectral" <br>(default),<br>"QA", "SnowIce", "Spectral Indices",<br> "Surface Reflectance", "Thermal",<br> "Top of Atmosphere Reflectance",<br> "Vegetation", "Water",<br> "Surface Temperature"<br><br>----------------------------------------<br>"All Bands", "Cirrus",<br> "Cloud","Landcover",<br> "Multispectral"(default),<br>"Panchromatic", "Pansharpen",<br> "Pansharpen and Multispectral",<br>"QA","SnowIce", "Thermal",<br> "Vegetation", "Water" <br><br>----------------------------------------<br> "Surface Reflectance"<br><br>----------------------------------------<br> "Spectral Indices"<br><br>----------------------------------------<br>"Top of Atmosphere Reflectance"<br><br>----------------------------------------<br>"Brightness Temperature"<br><br>----------------------------------------<br>"Surface Temperature"|
| "Landsat 1-5 MSS" | - | "All" (default),<br> "Level1" | "All Bands", <br>"Multispectral" (default),<br> "Pseudocolor", "Thermal",<br> "QA"
| "Sentinel-2" | "0.85, 0.7, 0.35, 1" | <br><br><br> "ALL" (default)<br><br><br><br><br><br>----------------------------------------<br>"Level1"<br><br><br><br><br><br>----------------------------------------<br>"Level2" | "All Bands",<br> "BOA Reflectance-10m",<br> "BOA Reflectance-20m",<br> "BOA Reflectance-60m",<br> "Multispectral" (default),<br> "Multispectral-10m",<br> "Multispectral-20m",<br> "Multispectral-60m",<br> "True Color", "SCL-20m",<br> "SCL-60m",<br> "Aerosol Optical Thickness",<br> "Water Vapour",<br> "BOA Reflectance" <br><br>----------------------------------------<br> "Multispectral" (default),<br> "Multispectral-10m",<br> "Multispectral-20m",<br> "Multispectral-60m",<br> "True Color" <br><br>----------------------------------------<br>"BOA Reflectance-10m",<br> "BOA Reflectance-20m",<br> "BOA Reflectance-60m",<br> "Aerosol Optical Thickness",<br> "Water Vapour",<br> "BOA Reflectance"
| "GeoEye-1" | "0.41, 0.16, 0.13, 0.3" | <br><br><br>"All" (default),<br> "Basic",<br> "Geo",<br> "GeoProfessional" "Orthorectified",<br> "Standard",<br> "Standard Orthoready" <br><br>----------------------------------------<br> "Orthorectified Acomp",<br> "Standard AComp",<br> "Standard OrthoreadyAcomp"| "All Bands",<br> "Multispectral",<br> "Multispectral AComp",<br> "Panchromatic",<br> "Panchromatic AComp",<br> "Pansharpen",<br> "Pansharpen AComp",<br> "Pansharpen and Multispectral" <br>(default) <br><br>----------------------------------------<br> "Multispectral AComp" (default),<br> "Panchromatic AComp",<br> "Pansharpen AComp"|
| "Pleiades-1" | "0.9, 0.75, 0.5, 0.5" | "All" (default),<br> "ORTHO",<br> "PRIMARY",<br> "PROJECTED"<br><br><br><br><br><br><br><br><br><br>----------------------------------------<br> "ORTHO DISPLAY" <br><br><br><br>----------------------------------------<br>"ORTHO REFLECTANCE" <br><br><br><br>----------------------------------------<br>"PRIMARY DISPLAY" <br><br><br><br>----------------------------------------<br> "PRIMARY REFLECTANCE" <br><br><br><br>----------------------------------------<br> "PROJECTED DISPLAY" <br><br><br><br>----------------------------------------<br> "PROJECTED REFLECTANCE" | <br><br>"All Bands",<br> "Multispectral",<br> "Multispectral Display",<br> "Multispectral Reflectance",<br> "Panchromatic",<br> "Panchromatic Display",<br> "Panchromatic Reflectance",<br> "Pansharpen",<br> "Pansharpen and Multispectral"<br> (default),<br> "Pansharpen Display",<br> "Pansharpen Reflectance" <br><br>----------------------------------------<br> "Multispectral Display" (default),<br> "Panchromatic Display",<br> "Pansharpen Display" <br><br>----------------------------------------<br> "Multispectral Reflectance" (default),<br> "Panchromatic Reflectance",<br> "Pansharpen Reflectance" <br><br>----------------------------------------<br> "Multispectral Display" (default),<br> "Panchromatic Display",<br> "Pansharpen Display" <br><br>----------------------------------------<br> "Multispectral Reflectance" (default),<br> "Panchromatic Reflectance",<br> "Pansharpen Reflectance" <br><br>----------------------------------------<br> "Multispectral Display" (default),<br> "Panchromatic Display",<br> "Pansharpen Display" <br><br>----------------------------------------<br> "Multispectral Reflectance" (default),<br> "Panchromatic Reflectance",<br> "Pansharpen Reflectance" |
| "SPOT 5" | "0.166,0.167,0.167,0.5" | "All" (default),<br> "SPOT Scene 1A",<br> "SPOT Scene 2A",<br> "SPOTView Ortho" | "All Bands",<br> "Multispectral",<br> "Panchromatic",<br> "Pansharpen",<br> "Pansharpen and Multispectral" <br>(default),<br> "Pseudocolor" |
| "SPOT 6" | "0.45, 0.55, 0, 0" | "All" (default),<br> "ORTHO", "PRIMARY",<br> "PROJECTED"<br><br><br><br><br><br><br><br><br>----------------------------------------<br> "ORTHO DISPLAY" <br><br><br><br>----------------------------------------<br>"ORTHO REFLECTANCE" <br><br><br><br>----------------------------------------<br>"PRIMARY DISPLAY" <br><br><br><br>----------------------------------------<br> "PRIMARY REFLECTANCE" <br><br><br><br>----------------------------------------<br> "PROJECTED DISPLAY" <br><br><br><br>----------------------------------------<br> "PROJECTED REFLECTANCE" | "All Bands",<br> "Multispectral",<br> "Multispectral Display",<br> "Multispectral Reflectance",<br> "Panchromatic",<br> "Panchromatic Display",<br> "Panchromatic Reflectance",<br> "Pansharpen",<br> "Pansharpen and Multispectral"<br> (default),<br> "Pansharpen Display",<br> "Pansharpen Reflectance" <br><br>----------------------------------------<br>"Multispectral Display" (default),<br> "Panchromatic Display",<br> "Pansharpen Display"<br><br>----------------------------------------<br> "Multispectral Reflectance" (default),<br> "Panchromatic Reflectance",<br> "Pansharpen Reflectance" <br><br>----------------------------------------<br> "Multispectral Display" (default),<br> "Panchromatic Display",<br> "Pansharpen Display" <br><br>----------------------------------------<br> "Multispectral Reflectance" (default),<br> "Panchromatic Reflectance",<br> "Pansharpen Reflectance" <br><br>----------------------------------------<br>"Multispectral Display" (default),<br> "Panchromatic Display",<br> "Pansharpen Dispplay" <br><br>----------------------------------------<br> "Multispectral Reflectance" (default),<br> "Panchromatic Reflectance",<br> "Pansharpen Reflectance" |
| "SPOT 7" | "0.45, 0.55, 0, 0" | <br><br>"All" (default),<br> "ORTHO",<br> "PRIMARY",<br> "PROJECTED"<br><br><br><br><br><br>----------------------------------------<br> "ORTHO DISPLAY" <br><br><br><br>----------------------------------------<br>"ORTHO REFLECTANCE" <br><br><br><br>----------------------------------------<br>"PRIMARY DISPLAY" <br><br><br><br>----------------------------------------<br> "PRIMARY REFLECTANCE" <br><br><br><br>----------------------------------------<br> "PROJECTED DISPLAY" <br><br><br><br>----------------------------------------<br> "PROJECTED REFLECTANCE" | "All Bands",<br> "Multispectral",<br> "Multispectral Display",<br> "Multispectral Reflectance",<br> "Panchromatic",<br> "Panchromatic Display",<br> "Panchromatic Reflectance",<br> "Pansharpen",<br> "Pansharpen and Multispectral"<br> (default),<br> "Pansharpen Display",<br> "Pansharpen Reflectance"<br><br>----------------------------------------<br> "Multispectral Display" (default),<br> "Panchromatic Display",<br> "Pansharpen Display" <br><br>----------------------------------------<br> "Multispectral Reflectance" (default),<br> "Panchromatic Reflectance",<br> "Pansharpen Reflectance" <br><br>----------------------------------------<br> "Multispectral Display" (default),<br> "Panchromatic Display",<br> "Pansharpen Display" <br><br>----------------------------------------<br> "Multispectral Reflectance" (default),<br> "Panchromatic Reflectance",<br> "Pansharpen Reflectance" <br><br>----------------------------------------<br> "Multispectral Display" (default),<br> "Panchromatic Display",<br> "Pansharpen Dispplay" <br><br>----------------------------------------<br> "Multispectral Reflectance" (default),<br> "Panchromatic Reflectance",<br> "Pansharpen Reflectance" |
| "IKONOS" | "0.378, 0.211, 0, 0.411" | "All" (default),<br> "Geo",<br> "GeoProfessional" | "All Bands",<br> "Multispectral",<br> "Panchromatic",<br> "Pansharpen",<br> "Pansharpen and Multispectral" <br>(default) |
| "WorldView-1" | - | "All" (default), "Basic",<br> "Orthorectified",<br> "Standard",<br> "Standard Orthoready" | "Panchromatic" (default) |
| "WorldView-2" | "0.39, 0.23, 0.21, 0.17" | <br><br>"All" (default),<br> "Basic",<br> "Orthorectified",<br> "Standard Orthoready",<br> "Standard"<br><br><br>----------------------------------------<br>"Orthorectified Acomp",<br> "Standard AComp",<br> "Standard OrthoreadyAcomp" | "All Bands",<br> "Multispectral",<br> "Multispectral AComp" "Panchromatic",<br> "Panchromatic AComp",<br> "Pansharpen AComp",<br> "Pansharpen",<br> "Pansharpen and Multispectral"<br> (default)<br><br>----------------------------------------<br>"Multispectral AComp" (default),<br> "Panchromatic AComp",<br> "Pansharpen AComp" |
| "WorldView-3" | "0.38, 0.25, 0.2, 0.16" |<br><br> "All" (default),<br> "Basic",<br> "Orthorectified",<br> "Standard Orthoready",<br> "Standard" <br><br><br><br>----------------------------------------<br>"Orthorectified Acomp",<br> "Standard AComp",<br> "Standard OrthoreadyAcomp"  | "All Bands",<br> "Multispectral",<br> "Multispectral AComp" "Panchromatic" ,<br> "Panchromatic AComp",<br> "Pansharpen",<br> "Pansharpen AComp",<br> "Pansharpen and Multispectral" (default),<br> "SWIR" <br><br><br>----------------------------------------<br>"Multispectral AComp" (default),<br> "Panchromatic AComp",<br> "Pansharpen AComp"|
| "WorldView-4" | "0.39, 0.23, 0.21, 0.17" | "All" (default), "Basic",<br> "Standard OrthoReady" | "All Bands" "Multispectral",<br> "Panchromatic" "Pansharpen" ,<br> "Pansharpen and Multispectral"<br> (default),<br> "Multispectral AComp",<br> "Panchromatic AComp",<br> "Pansharpen AComp" |
| "ZY3-SASMAC" | "0.329, 0.509, 0, 0.162" | "All" (default),<br> "LEVEL1" | "All Bands" "Double Line Stereo",<br> "Multispectral",<br> "Panchromatic",<br> "Panchromatic BWD",<br> "Panchromatic FWD",<br> "Panchromatic NAD",<br> "Pansharpen"<br> "Pansharpen and <br>Multispectral" (default),<br> "Three Line Stereo" |
| "Raster Dataset" | - | - | -
| "NetCDF" | - | "All" (default) | "Default"(default),<br> "Multiband Composite",<br> "Vector Field" |
| "GRIB" | - | "All" (default) | "Default"(default),<br> "Multiband Composite",<br> "Vector Field" |
| "HDF" | - | "All" (default) | "Default"(default),<br> "Multiband Composite",<br> "Vector Field" |
| "ASTER" | - | "All" (default),<br> "L1T",<br> "L1B",<br> "Surface Reflectance",<br> "Surface Radiance",<br> "Surface Emissivity",<br> "Surface Kinetic Temperature",<br> "DEM", "Registered Radiance" | "VNIR" (default),<br> "SWIR",<br> "TIR",<br> "Multispectral" (default),<br> "TIR - Surface Emissivity",<br> "VNIR - Surface Reflectance",<br> "SWIR - Surface Reflectance",<br> "TIR - Surface kinetic Temperature",<br> "VNIR - Surface Radiance",<br> "SWIF - Surface Radiance",<br> "TIR - Surface Radiance",<br> "VNIR - Registered Radiance",<br> "VNIR - Registered Radiance",<br> "SWIR - Registered Radiance",<br> "TIR - Registered Radiance",<br> "Global Digital Elevation Model (ASTGTM)",<br> "Digital Elevation Model",<br> "All Bands" |
| "DMCii" | - | "All" (default),<br> "L1R",<br> "L1T" | "Multispectral" (default),<br> "ALL",<br> "Pseudocolor" |
| "RapidEye" | - | "All",<br> "Level1B",<br> "Level3A",<br> "Level3B" | "Multispectral" (default) | 
| "SkySat" | "0.41, 0.16, 0.13, 0.3" | "All",<br> "Basic",<br> "Ortho" | "All Bands",<br> "Multispectral",<br> "Panchromatic",<br> "Pansharpen" ,<br> "All Bands",<br> "Skysat Pansharpen",<br> "Visual",<br> "Pansharpen and Multispectral"<br> (default) |